<a href="https://colab.research.google.com/github/JonathanCecil01/ContextSearch/blob/main/context_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy

In [ ]:
!pip install PyPDF2

In [ ]:
!python -m spacy download en_core_web_lg

In [32]:
import spacy
from PyPDF2 import PdfReader
import json

In [5]:
nlp = spacy.load('en_core_web_lg')

In [6]:
def extract_text_from_pdf(path):
    # creating a pdf reader object
    reader = PdfReader(path)

    # printing number of pages in pdf file
    print(len(reader.pages))

    # getting a specific page from the pdf file
    text_array = []


    for i in range(20, len(reader.pages)-20):
    # extracting text from page
        page = reader.pages[i]
        text = page.extract_text()
        text_array.append(text)

    return text_array

In [28]:
# Process each page using spaCy

def sentence_extraction(filename):
  text_array =  extract_text_from_pdf(filename)
  processed_pages = []
  page_no = 0
  for page in text_array:
      doc = nlp(page)
      processed_pages.append([doc, page_no])
      page_no+=1


  sentences = []
  for doc, page_no in processed_pages:
      for sentence in doc.sents:
          sentences.append([sentence, page_no])
  return sentences


In [34]:
def create_json_file(context_sents, filename):
  sentences = sentence_extraction(filename)
  context_dict = {}
  for input_sent in context_sents:
    context_dict[input_sent] = []

  for input_sent in context_sents:
    # Process the input sentence
    input_doc = nlp(input_sent)

    # Calculate similarity with each sentence
    similarity_scores = []
    for sentence, page_no in sentences:
        similarity_score = input_doc.similarity(sentence)
        similarity_scores.append((sentence, similarity_score, page_no))

    # Sort similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    i = 0
    # Print the most similar sentences
    for sentence, similarity_score, page_no in similarity_scores:
        temp_dict = {}
        temp_dict['file'] = filename
        temp_dict['page_no'] = page_no
        temp_dict['sentence'] = sentence.text
        temp_dict['similarity_score'] = similarity_score
        context_dict[input_sent].append(temp_dict)
        if i==10:
          break
        i+=1

    print(context_dict)

    json_string = json.dumps(context_dict, indent = 2)
    with open("context_search_results.json", "w") as f:
      f.write(json_string)



In [36]:
create_json_file(["neural networks are the best for image recognition", "RNN is difficult", "Learning Word embeddings", "Transformers are just complex networks"], '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf')

512


<ipython-input-34-ebdc217cd53d>:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score = input_doc.similarity(sentence)


{'neural networks are the best for image recognition': [{'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 199, 'sentence': 'Convolutional neural networks are used for image\nrecognition and prediction.', 'similarity_score': 0.9343882001078312}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 39, 'sentence': '1.6.5 Convolutional Neural Networks\nConvolutional neural networks are biologically inspired networks that are used in computer\nvision for image classiﬁcation and object detection.', 'similarity_score': 0.9264531609446556}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 366, 'sentence': 'The two most popular data sets for testing convolutional neural networks are MNIST\nandImageNet .', 'similarity_score': 0.9095713538369533}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 52, 'sentence': 'Neural networks are also parameterized models that are learned with continuous opti